In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Sequential
from tensorflow.keras import  models

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers as L
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
IMG_SHAPE=(224,224)


In [ ]:
def prepare_data_to_model(data_dir: str,shape=(224,224),class_mode='categorical',batch_size=32,shuffle=True):
    train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)
    train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=shape,
    batch_size=batch_size,
    class_mode=class_mode,
    shuffle=shuffle)
    return train_generator

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):

        if(logs.get('accuracy') > 0.98):
            # Stop if threshold is met
            print("\naccuracy of training is bigger than 98!")
            self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

In [ ]:
train_generator=tf.keras.preprocessing.image_dataset_from_directory(directory='/content/drive/MyDrive/plate-classification',
                                                                            image_size=(120,224),
                                                                            label_mode="categorical", # what type are the labels?
                                                                            batch_size=32,
                                                                            shuffle=True)
val_generator=tf.keras.preprocessing.image_dataset_from_directory(directory='/content/drive/MyDrive/plate-classification-val',
                                                                            image_size=(120,224),
                                                                            label_mode="categorical", # what type are the labels?
                                                                            batch_size=32,
                                                                            shuffle=True)


Found 432 files belonging to 8 classes.
Found 96 files belonging to 8 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=8)

checkpoint_path='/content/callbacks'
!mkdir {checkpoint_path}
checkpoint= ModelCheckpoint(
            checkpoint_path,
            save_weights_only=True, # save only the model weights
            monitor="accuracy", # save the model weights which score the best validation accuracy
            save_best_only=True)


In [ ]:
train_generator=prepare_data_to_model('/content/drive/MyDrive/plate-classification')
classes=list(train_generator.class_indices.keys())

Found 432 images belonging to 8 classes.


In [ ]:
data_augmentation = Sequential([
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomWidth(0.1),
    preprocessing.RandomHeight(0.1),
    preprocessing.RandomFlip("vertical"),
    preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.RandomBrightness(
    0.4, value_range=(0, 255), seed=None, ),
    tf.keras.layers.RandomTranslation(
    0.1,
    0.2,
    fill_mode='reflect',
    interpolation='bilinear',
    seed=None,
    fill_value=0.0,
),
    tf.keras.layers.RandomContrast(0.3, seed=None,),
    preprocessing.RandomRotation(0.2),
    # حرك النمره لفوق او في الجمب
    # preprocessing.RandomShear((0.3,0.3)), # move each pixel in any direction by 40 pixel
    # preprocessing.RandomChannelShift((20,20,20)), # change rgb values by 20 up or down
    # preprocessing.RandomBrightness(0.4), # change the light
    # preprocessing.RandomGamma((0.1,0.4)),# change light and contrast fron 0.1 to 0.4
    # preprocessing.RandomCrop(height=224, width=224),
], name='data-augmentiation-layer')

In [ ]:
model_e= ResNet50(include_top=False)
model_e.trainable=False
input_layer= L.Input(shape=(120, 224, 3),name='input_layer')
x= data_augmentation(input_layer)
x=model_e(x)
x=L.GlobalAveragePooling2D(name='minimazie_dimentions')(x)
x= L.Dense(128,activation='relu',name='added_layer_1')(x)
x= L.Dense(64,activation='relu',name='added_layer_2')(x)
output_layer= L.Dense(len(classes),activation='softmax',name='output_layer')(x)
model= keras.Model(input_layer,output_layer)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
history=model.fit(train_generator,
                  validation_data=val_generator,
             epochs=33,
          callbacks=[
              checkpoint,
              early_stopping
          ])

In [ ]:
for i in model.layers:
    print(i.name.ljust(25),i.trainable)


input_layer               True
data-augmentiation-layer  True
resnet50                  False
minimazie_dimentions      True
added_layer_1             True
added_layer_2             True
output_layer              True


In [ ]:
for layer in model.layers[len(model.layers)-16:]:
  layer.trainable = True

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=["accuracy"])

In [ ]:
model.fit(              train_generator,
                        epochs=history.epoch[-1]+55,
                        validation_data=val_generator,
                        initial_epoch=history.epoch[-1],
          callbacks=[callbacks])

Epoch 17/71


14/14 [==============================] - 60s 1s/step - loss: 0.7051 - accuracy: 0.7153
Epoch 18/71
14/14 [==============================] - 15s 1s/step - loss: 0.5815 - accuracy: 0.7500
Epoch 19/71
14/14 [==============================] - 13s 934ms/step - loss: 0.5095 - accuracy: 0.7708
Epoch 20/71
14/14 [==============================] - 12s 833ms/step - loss: 0.3823 - accuracy: 0.8310
Epoch 21/71
14/14 [==============================] - 12s 892ms/step - loss: 0.4355 - accuracy: 0.8125
Epoch 22/71
14/14 [==============================] - 13s 890ms/step - loss: 0.3964 - accuracy: 0.8056
Epoch 23/71
14/14 [==============================] - 11s 787ms/step - loss: 0.3868 - accuracy: 0.8079
Epoch 24/71
14/14 [==============================] - 12s 840ms/step - loss: 0.3904 - accuracy: 0.8102
Epoch 25/71
14/14 [==============================] - 11s 788ms/step - loss: 0.3274 - accuracy: 0.8472
Epoch 26/71
14/14 [==============================] - 11s 806ms/step - loss: 0.4206 - accuracy: 0.812

In [ ]:
model.save('plate_classifaction_resnet.h5')

In [ ]:
import cv2
def classifier(path,model,shape)-> int:
    img=cv2.imread(path)
    img=cv2.resize(img,dsize=shape)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    # img=preprocess_input(img_rgb)
    img=np.expand_dims(img,axis=0)
    pred=model.predict(img,verbose=0)
    idx=np.argmax(pred)
    print('model output: ',pred)

    # values={k:i for i,k in train_batches.class_indices.items()}
    # print(values[idx])
    return idx,pred

In [ ]:
x='/content/0001_license_plate_1.png'
i,p=classifier(x,model,(224,224))
print(classes[i],i)
print(list(p[0]))
plt.imshow(x)

In [ ]:
# classes
train_generator[0]/255.